In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB
from scipy.stats import multivariate_normal as mvn
from sklearn.feature_selection import VarianceThreshold
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score

**1-Entrenamiento**

Carga de los datos de entrenamiento 

In [71]:
x_train = pd.read_csv('practica_X_train.csv', sep=';', index_col=0)
y_train = pd.read_csv('practica_Y_train.csv', sep=';', index_col=0)

# Dividir los datos en conjuntos de entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Verificar las dimensiones de los conjuntos
print("Tamaño x_train:", x_train.shape)
print("Tamaño x_test:", x_val.shape)

print(x_train.head())
print(y_train.head())

#recomendado hacer k_fold para conjunto de entrenamiento y validacion

Tamaño x_train: (3200, 9)
Tamaño x_test: (800, 9)
      Temperature  Humidity  PM2.5  PM10  NO2  SO2   CO  \
2386          5.0       6.0    3.0   4.0  6.0  4.0  5.0   
4737          2.0       2.0    0.0   1.0  3.0  1.0  0.0   
1153          2.0       2.0    0.0   0.0  3.0  2.0  1.0   
3840          5.0       5.0    0.0   1.0  4.0  1.0  7.0   
4229          3.0       3.0    0.0   1.0  5.0  4.0  4.0   

      Proximity_to_Industrial_Areas  Population_Density  
2386                            1.0                 5.0  
4737                            3.0                 4.0  
1153                            3.0                 2.0  
3840                            2.0                 4.0  
4229                            3.0                 2.0  
     Air_Quality
2386   Hazardous
4737        Good
1153        Good
3840        Poor
4229        Poor


1.1 Preprocesado ejemplos

Como el conjunto de datos x_train son datos continuos y en el enunciado se nos dice que no tenemos valores perdidos y en los modelos de Naive Bayes, MVN y GMM los datos de tipo continuo son buenas entradas. 

Por lo tanto, el único preprocesado que sería necesario para poder trabajar con el conjunto de test es codificar el atributo categórico del conjunto de y_train y aplicar un filtrado por varianza al conjunto de datos de x_train para descartar aquellas columnas que no aporten información. 

In [72]:
#filtrado por varianza conjunto x_train
thereshold = 0.1

sel = VarianceThreshold(threshold=thereshold)
sel.set_output(transform='pandas')
x_train = sel.fit_transform(x_train)

# y de x_val
x_val = sel.transform(x_val)

In [73]:
#codificación conjunto y_train
categoria = y_train['Air_Quality'].unique()
categoria_codificada ={string:i for i,string in enumerate(categoria)} #diccionario de codificación
cat = y_train['Air_Quality'].map(categoria_codificada)

y_train['Air_Quality'] = cat
print(y_train.head())

y_val['Air_Quality'] = y_val['Air_Quality'].map(categoria_codificada)

      Air_Quality
2386            0
4737            1
1153            1
3840            2
4229            2


1.2 Construcción modelo a priori de la etiqueta (cada nivel de calidad del aire tiene un número diferente de ejemplos)

In [74]:
clases = y_train['Air_Quality'].value_counts()  # Cantidad de todas las soluciones distintas
Probs_priori = clases / len(y_train)
print("Probabilidades a priori: ")
print(Probs_priori)

Probabilidades a priori: 
Air_Quality
1    0.399375
3    0.296563
2    0.206250
0    0.097812
Name: count, dtype: float64


1.3 Construir distintos modelos de verosimilitud

Naive Bayes 

Primer modelo posible con GaussianNB

In [75]:
naive_bayes_gaussian = GaussianNB()
naive_bayes_gaussian.fit(x_train, y_train.values.ravel()) # con y_train.values.ravel() 
                                                 # cambiamos de lista pandas a numpy
                                                 # Es para que solo nos den los valores 
                                                 # Sino te da un enlace raro tambien
print("Probabilidades a priori con Naive Bayes Gassiano:") 
print(naive_bayes_gaussian.class_prior_) # Solo para comprobar las probabilidades de las clases

predicciones_g = naive_bayes_gaussian.predict(x_train)
predicciones_g_val = naive_bayes_gaussian.predict(x_val)
cont = 0
for y_hat, y_true in zip(predicciones_g, y_train.values.ravel()):
    if (y_true == y_hat):
        cont += 1

acc_train_g = accuracy_score(y_train, predicciones_g)
acc_val_g = accuracy_score(y_val, predicciones_g_val)
print('acc train:', acc_train_g)
print('acc test', acc_val_g)



Probabilidades a priori con Naive Bayes Gassiano:
[0.0978125 0.399375  0.20625   0.2965625]
acc train: 0.9178125
acc test 0.9275


Segundo modelo posible con CategoricalNB

In [76]:
from sklearn.preprocessing import KBinsDiscretizer

# Discretizar en bins
discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')  # o 'quantile' si prefieres
discretizer.set_output(transform='pandas')

x_train_cat = discretizer.fit_transform(x_train)
x_val_cat = discretizer.transform(x_val)


naive_bayes_categorical = CategoricalNB()
naive_bayes_categorical.fit(x_train_cat, y_train.values.ravel())

predicciones_c = naive_bayes_categorical.predict(x_train_cat)
predicciones_c_val = naive_bayes_categorical.predict(x_val_cat)
cont = 0
for y_hat, y_true in zip(predicciones_c, y_train.values.ravel()):
    if (y_true == y_hat):
        cont += 1
        

acc_train_c = accuracy_score(y_train, predicciones_c)
acc_val_c = accuracy_score(y_val, predicciones_c_val)
print('acc train:', acc_train_c)
print('acc test', acc_val_c)


acc train: 0.9490625
acc test 0.9575


Tercer modelo posible BernoulliNB

In [77]:
naive_bayes_bernoulli = BernoulliNB(alpha= 1)
naive_bayes_bernoulli.fit(x_train, y_train.values.ravel())

predicciones_b = naive_bayes_bernoulli.predict(x_train)
predicciones_b_val = naive_bayes_bernoulli.predict(x_val)
cont = 0
for y_hat, y_true in zip(predicciones_b, y_train.values.ravel()):
    if (y_true == y_hat):
        cont += 1
        
acc_train_b = accuracy_score(y_train, predicciones_b)
acc_val_b = accuracy_score(y_val, predicciones_b_val)
print('acc train:', acc_train_c)
print('acc test', acc_val_c)

acc train: 0.9490625
acc test 0.9575


MVN

In [78]:
def train_mvn(Y_TRAIN, X_TRAIN):
    estadisticas_modelo = {}
    clases = np.unique(Y_TRAIN)
    for clase in clases:
        datos_clase = X_TRAIN[Y_TRAIN == clase]

        media = np.mean(datos_clase, axis=0)
        covarianza = np.cov(datos_clase, rowvar=False)
        covarianza += np.eye(covarianza.shape[0]) * 1e-6

        modelo = mvn(mean=media, cov=covarianza)

        estadisticas_modelo[clase] = {"media": media, "covarianza": covarianza, "modelo": modelo}
    return estadisticas_modelo

def predict_mvn(x_a_predecir, estadisticas):
    # Predecimos probabilidades primero
    probs = []
    clases = list(estadisticas.keys())
    for clase in clases:
        modelo = estadisticas[clase]["modelo"]
        p = modelo.logpdf(x_a_predecir)
        probs.append(p)
    probs = np.array(probs).T
    y_hat = np.argmax(probs, axis=1)
    return np.array(clases)[y_hat]
    

In [79]:
y_train_flat = y_train.values.ravel()

estadisticas_MVN = train_mvn(y_train_flat, x_train.values)

y_hat = predict_mvn(x_train.values, estadisticas_MVN)
acc = np.mean(y_hat == y_train_flat)
print("acc con conjunto train: ", acc)

acc con conjunto train:  0.924375


GMM

In [80]:
# Calcular las medias de cada clase
means_class_0 = x_train[y_train_flat == 0].mean(axis=0).values.reshape(1, -1)
means_class_1 = x_train[y_train_flat == 1].mean(axis=0).values.reshape(1, -1)

# Iniciar los parámetros de los modelos GMM con las medias de cada clase
means_init_0 = np.tile(means_class_0, (4, 1))
means_init_1 = np.tile(means_class_1, (4, 1))

# Entrenar el modelo GMM para cada clase
gmm0 = GaussianMixture(n_components=4, means_init=means_init_0, covariance_type='diag', max_iter=10)
gmm0.fit(x_train[y_train_flat == 0])

gmm1 = GaussianMixture(n_components=4, means_init=means_init_1, covariance_type='diag', max_iter=10)
gmm1.fit(x_train[y_train_flat == 1])

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/mixture/_base.py:269: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(


GaussianMixture(covariance_type='diag', max_iter=10,
                means_init=array([[2.11658842, 2.08841941, 0.1314554 , 0.18309859, 1.60172144,
        1.69953052, 0.66353678, 3.6885759 , 2.26995305],
       [2.11658842, 2.08841941, 0.1314554 , 0.18309859, 1.60172144,
        1.69953052, 0.66353678, 3.6885759 , 2.26995305],
       [2.11658842, 2.08841941, 0.1314554 , 0.18309859, 1.60172144,
        1.69953052, 0.66353678, 3.6885759 , 2.26995305],
       [2.11658842, 2.08841941, 0.1314554 , 0.18309859, 1.60172144,
        1.69953052, 0.66353678, 3.6885759 , 2.26995305]]),
                n_components=4)

In [81]:
#Ver lo bueno que es el GMM, ver el ajuste por cada clase 
log_likelihood = gmm0.score(x_train)
print("Log-Verosimilitud clase 0: ", log_likelihood)

log_likelihood = gmm1.score(x_train)
print("Log-Verosimilitud clase 1: ", log_likelihood)

Log-Verosimilitud clase 0:  -27.834670537906785
Log-Verosimilitud clase 1:  -32.4269207637457


1.4 Construir sistemas con preprocesado, modelo a priori y cada uno de los modelos de verosimilitud

Sistema con Naive Bayes 

In [82]:
def sistema_naive_bayes_categorical(x_train, x_val, y_train, y_val):
    # Filtrado por varianza
    threshold = 0.1
    sel = VarianceThreshold(threshold=threshold)
    sel.set_output(transform='pandas')
    x_train_pre = sel.fit_transform(x_train)
    x_val_pre = sel.transform(x_val)

    # Discretización para CategoricalNB
    discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
    discretizer.set_output(transform='pandas')
    x_train_discretized = discretizer.fit_transform(x_train_pre)
    x_val_discretized = discretizer.transform(x_val_pre)

    # Codificación de y
    categorias = y_train['Air_Quality'].unique()
    categoria_codificada = {string: i for i, string in enumerate(categorias)}
    y_train_cod = y_train['Air_Quality'].map(categoria_codificada)
    y_val_cod = y_val['Air_Quality'].map(categoria_codificada)

    # Modelo
    naive_bayes_categorical = CategoricalNB()
    naive_bayes_categorical.fit(x_train_discretized, y_train_cod)

    # Predicción y evaluación
    pred_val = naive_bayes_categorical.predict(x_val_discretized)
    acc_val = np.mean(pred_val == y_val_cod)
    print("Accuracy con conjunto validación:", acc_val)

    return naive_bayes_categorical, sel, discretizer, categoria_codificada

Sistema con MVN

In [83]:
def sistema_mvn(x_train, x_val, y_train, y_val):
    # Filtrado por varianza
    threshold = 0.1
    sel = VarianceThreshold(threshold=threshold)
    sel.set_output(transform='pandas')
    x_train_pre = sel.fit_transform(x_train)
    x_val_pre = sel.transform(x_val)

    # Codificación de y
    categorias = y_train['Air_Quality'].unique()
    categoria_codificada = {string: i for i, string in enumerate(categorias)}
    y_train_cod = y_train['Air_Quality'].map(categoria_codificada)
    y_val_cod = y_val['Air_Quality'].map(categoria_codificada)

    # Modelo
    modelo_mvn = train_mvn(y_train_cod.values, x_train_pre.values)

    # Evaluación en validación
    pred_val = predict_mvn(x_val_pre.values, modelo_mvn)
    acc_val = np.mean(pred_val == y_val_cod.values)
    print("Accuracy con conjunto validación:", acc_val)

    return modelo_mvn, sel, categoria_codificada


Sistema con GMM

In [84]:
def sistema_gmm(x_train, x_val, y_train, y_val):
    # Filtrado por varianza
    threshold = 0.1
    sel = VarianceThreshold(threshold=threshold)
    sel.set_output(transform='pandas')
    x_train_pre = sel.fit_transform(x_train)
    x_val_pre = sel.transform(x_val)

    # Codificación de y
    categorias = y_train['Air_Quality'].unique()
    categoria_codificada = {string: i for i, string in enumerate(categorias)}
    y_train_cod = y_train['Air_Quality'].map(categoria_codificada).values
    y_val_cod = y_val['Air_Quality'].map(categoria_codificada).values

    # Probabilidades a priori
    clases, counts = np.unique(y_train_cod, return_counts=True)
    probs_priori = {clase: count / len(y_train_cod) for clase, count in zip(clases, counts)}

    # Entrenamiento GMM
    n_components=4
    modelos_gmm = {}
    for clase in clases:
        datos_clase = x_train_pre[y_train_cod == clase]
        media_clase = datos_clase.mean(axis=0).values.reshape(1, -1)
        means_init = np.tile(media_clase, (n_components, 1))

        gmm = GaussianMixture(
            n_components=n_components,
            means_init=means_init,
            covariance_type='diag',
            max_iter=100,
            random_state=42
        )
        gmm.fit(datos_clase)
        modelos_gmm[clase] = gmm

    # Predicción en validación
    log_probs = []
    for clase in clases:
        log_likelihood = modelos_gmm[clase].score_samples(x_val_pre)
        log_prior = np.log(probs_priori[clase])
        log_probs.append(log_likelihood + log_prior)

    pred_val = np.argmax(np.vstack(log_probs).T, axis=1)
    acc_val = np.mean(pred_val == y_val_cod)
    print("Accuracy con conjunto validación:", acc_val)

    return modelos_gmm, sel, categoria_codificada


1.5 Comparar los sistema con un conjunto de validación  (elegir uno de los sistemas)

In [85]:
modelo, selector_varianza, dis ,codificacion_categorias = sistema_naive_bayes_categorical(x_train, x_val, y_train, y_val)
modelo_mvn, selector_varianza_mvn, codificacion_categorias_mvn = sistema_mvn(x_train, x_val, y_train, y_val)
modelo_gmm, selector_varianza_gmm, codificacion_categorias_gmm = sistema_gmm(x_train, x_val, y_train, y_val)

Accuracy con conjunto validación: 0.9575
Accuracy con conjunto validación: 0.93375
Accuracy con conjunto validación: 0.81625


1.6 Almacenamiento objetos con Pickle

In [ ]:
import pickle

# Guardar solo el modelo y los tratamientos de x_train
with open('modelo_y_tratamientos.pkl', 'wb') as f:
    pickle.dump({
        'modelo': naive_bayes_categorical,
        'selector_varianza': sel,
        'discretizador': discretizer
    }, f)

print("Modelo y tratamientos de x_train guardados correctamente.")

Modelo y tratamientos de x_train guardados correctamente.
